In [1]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
train = pd.read_csv('/content/drive/MyDrive/DACON/train_features.csv')

In [5]:
target = pd.read_csv('/content/drive/MyDrive/DACON/train_labels.csv')
train_labels = target

In [6]:
test = pd.read_csv('/content/drive/MyDrive/DACON/test_features.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/sample_submission.csv')

In [7]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [8]:
train['id'].value_counts()

2047    600
1347    600
2378    600
1354    600
2379    600
       ... 
1699    600
675     600
2722    600
1698    600
0       600
Name: id, Length: 3125, dtype: int64

In [9]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [10]:
ex1 = train_labels[train_labels['label']==26]

In [11]:
idx_26 = train_labels[ (train_labels['label'] == 26 ) & (train_labels['id'] < 2042 ) ]

In [12]:
idx_26

,id,label,label_desc
1,1,26,Non-Exercise
3,3,26,Non-Exercise
4,4,26,Non-Exercise
5,5,26,Non-Exercise
6,6,26,Non-Exercise
...,...,...,...
2030,2030,26,Non-Exercise
2031,2031,26,Non-Exercise
2032,2032,26,Non-Exercise
2035,2035,26,Non-Exercise


In [13]:
train_dup = train

In [14]:
merge_test = pd.merge(train_dup , idx_26 , on="id" , how="outer")

In [15]:
train_ = merge_test[merge_test.label != 26.0]

In [16]:
train_

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label,label_desc
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,NaN,NaN
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,NaN,NaN
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,NaN,NaN
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,NaN,NaN
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,NaN,NaN
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,NaN,NaN
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,NaN,NaN
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,NaN,NaN


In [17]:
del train_['label']

In [18]:
del train_['label_desc']

In [19]:
train_

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [21]:
print(X.shape)

torch.Size([2142, 600, 6])


In [22]:
train_labels_ = train_labels
merge_test = pd.merge(train_labels_ , idx_26 , on="id" , how="outer")

In [23]:
merge_test

,id,label_x,label_desc_x,label_y,label_desc_y
0,0,37,Shoulder Press (dumbbell),NaN,NaN
1,1,26,Non-Exercise,26.0,Non-Exercise
2,2,3,Biceps Curl (band),NaN,NaN
3,3,26,Non-Exercise,26.0,Non-Exercise
4,4,26,Non-Exercise,26.0,Non-Exercise
...,...,...,...,...,...
3120,3120,26,Non-Exercise,NaN,NaN
3121,3121,26,Non-Exercise,NaN,NaN
3122,3122,15,Dynamic Stretch (at your own pace),NaN,NaN
3123,3123,26,Non-Exercise,NaN,NaN


In [24]:
merge_test = merge_test[merge_test['label_y'] != 26.0]

In [25]:
del merge_test['label_y']
del merge_test['label_desc_y']

In [26]:
train_labels_ = merge_test

In [27]:
train_labels_

,id,label_x,label_desc_x
0,0,37,Shoulder Press (dumbbell)
2,2,3,Biceps Curl (band)
12,12,39,Side Plank Right side
13,13,28,Plank
20,20,6,Chest Press (rack)
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [20]:
X = torch.FloatTensor(train_.iloc[:,2:].values).view(-1, 600, 6)

In [28]:
y = torch.FloatTensor(train_labels_['label_x'].values)

In [29]:
y.shape

torch.Size([2142])

In [30]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [31]:
y = y.view(-1, 1)

In [34]:
y.shape

torch.Size([2142, 1])

In [20]:
class Net(torch.nn.Module):
  def __init__(self , input_size , hidden_size):
    super(Net, self).__init__()
    self.lstm = torch.nn.LSTM(input_size , hidden_size , batch_first= True)
    self.fc1 = torch.nn.Linear(hidden_size , 128 , bias = True)
    self.relu = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(128,61 ,bias = True)

  def forward(self, x):
    x, _status = self.lstm(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [21]:
output = Net(6,61)

In [37]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(output.parameters(), 1e-3)

In [38]:
y = y.squeeze(1).long()

In [39]:
for i in range(4):
  optimizer.zero_grad()
  outputs = output(X)
  loss = criterion(outputs[:,-1,:] , y)
  loss.backward()
  optimizer.step()
  
  print('epoch : ',i, " loss : ", loss.item(), '\n')

epoch :  0  loss :  4.087735652923584 

epoch :  1  loss :  4.071544647216797 

epoch :  2  loss :  4.055592060089111 

epoch :  3  loss :  4.039753437042236 



In [28]:
with torch.no_grad():
  X_test = torch.FloatTensor(test.iloc[:,2:].values).view(-1, 600, 6)
  pred_test = output(X_test)

In [29]:
pd.DataFrame(torch.argmax(pred_test[:, -1, :], axis = 1))

,0
0,40
1,40
2,18
3,40
4,18
...,...
777,26
778,4
779,26
780,26
